<a href="https://colab.research.google.com/github/danielbauer1860/LDS_Project/blob/main/generation/generating_aca_2epoch_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This notebook references [the project of Salminen et al. (2021)](https://github.com/joolsa/FakeReviews/blob/)

In [2]:
!pip install iteround -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 tensorboard huggingface_hub[cli] xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.ca

# Loading the model

Redownloading the model required some tweaks, which are described in:
https://stackoverflow.com/questions/76459034/how-to-load-a-fine-tuned-peft-lora-model-based-on-llama-with-huggingface-transfo

In [5]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import infer_auto_device_map, init_empty_weights

peft_model_id = 'dbauer1860/llama-2-bnc-baby-academic-2-epochs'

config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    torch_dtype='auto',
    device_map='auto',
    offload_folder="offload", offload_state_dict = True
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

# Extracting Word Counts and Prompts

In [6]:
import pandas as pd
import numpy as np
import random
import nltk
from iteround import saferound
from nltk.tokenize import word_tokenize

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
df = pd.read_csv('/content/drive/MyDrive/Linguistic Data Science/data/bnc_baby_texts.csv', sep='|')

In [9]:
df.head()

,text,category
0,"Why do certain images matter to one, and why i...",ACA
1,"In many developing countries, the immunogenici...",ACA
2,Anthony Kenny The British Library 1992 Scholar...,ACA
3,Beforehand he worked as a probation officer fo...,ACA
4,CONTINUITY AND CHANGE are features of every po...,ACA


In [10]:
def split_into_sentences(text):
    return nltk.sent_tokenize(text)

In [11]:
df['sentences'] = df['text'].apply(split_into_sentences)

In [12]:
df

,text,category,sentences
0,"Why do certain images matter to one, and why i...",ACA,"[Why do certain images matter to one, and why ..."
1,"In many developing countries, the immunogenici...",ACA,"[In many developing countries, the immunogenic..."
2,Anthony Kenny The British Library 1992 Scholar...,ACA,[Anthony Kenny The British Library 1992 Schola...
3,Beforehand he worked as a probation officer fo...,ACA,[Beforehand he worked as a probation officer f...
4,CONTINUITY AND CHANGE are features of every po...,ACA,[CONTINUITY AND CHANGE are features of every p...
...,...,...,...
177,"Artworks THE Friends of Oriel Ynys Mon, Llange...",NEWS,"[Artworks THE Friends of Oriel Ynys Mon, Llang..."
178,THERE will be those who portray the report of ...,NEWS,[THERE will be those who portray the report of...
179,BUCKINGHAM PALACE 30 September: The Princess R...,NEWS,[BUCKINGHAM PALACE 30 September: The Princess ...
180,The Scottish Landowners' Federation unequivoca...,NEWS,[The Scottish Landowners' Federation unequivoc...


In [13]:
random_state = 21
random_sentence_state = 21
sample_count = 20

In [14]:
def get_random_sentence(sentences_list, random_state=random_sentence_state, min_word_count=5):
    random.seed(random_state)
    filtered_sentences = [sentence for sentence in sentences_list if len(word_tokenize(sentence)) > min_word_count]
    return random.choice(filtered_sentences) if filtered_sentences else None

In [15]:
df['prompt'] = df['sentences'].apply(get_random_sentence)

In [16]:
def get_samples(df, n=sample_count, state=random_state):
  return df.sample(n=n, random_state=state)

In [17]:
df_aca, df_dem, df_fic, df_news = [get_samples(y) for x, y in df.groupby(['category'])]

<ipython-input-17-c993aa370679>:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  df_aca, df_dem, df_fic, df_news = [get_samples(y) for x, y in df.groupby(['category'])]


In [18]:
df_news

,text,category,sentences,prompt
95,This week's figures suggest that one in three ...,NEWS,[This week's figures suggest that one in three...,It is less ambiguous than the previous version...
116,It won the contract to build the power take-of...,NEWS,[It won the contract to build the power take-o...,"Households in Wales spend most on tea, at 66p ..."
108,Opening with a tirade of four letter abuse fro...,NEWS,[Opening with a tirade of four letter abuse fr...,"Details, tel Mrs Hartshorne ."
131,‘What I have found in the region is that becau...,NEWS,[‘What I have found in the region is that beca...,They in turn work closely with local authoriti...
124,Morning sickness good for mothers By Roger Hig...,NEWS,[Morning sickness good for mothers By Roger Hi...,It involves nothing more than putting up 80-fo...
160,Interim figures unveiled yesterday show a 16% ...,NEWS,[Interim figures unveiled yesterday show a 16%...,Here are 10 New Year resolutions to get you in...
114,"The baby is the accessory of the moment, seen ...",NEWS,"[The baby is the accessory of the moment, seen...","Richard Holmes, the sales and marketing office..."
149,THERE was confusion yesterday about the affect...,NEWS,[THERE was confusion yesterday about the affec...,"on the year at about 140,000."
98,EVERYONE who grew up in the Sixties has heard ...,NEWS,[EVERYONE who grew up in the Sixties has heard...,The Maharishi Mahesh Yogi was a slickly-packag...
107,BIRMINGHAM'S infamous and exhausted retail hub...,NEWS,[BIRMINGHAM'S infamous and exhausted retail hu...,"From this generous lobby, looked down on by di..."


In [19]:
aca_prompts = df_aca['prompt'].to_list()
fic_prompts = df_fic['prompt'].to_list()
news_prompts = df_news['prompt'].to_list()

# Generating output

In [20]:
#Generation parameters; these were, for the most part, set in accordance to Salminen et al.
#the only difference is the implementation of a deviation value
do_sample = True
top_k = 0
top_p = 0.92
temperature = 0.7
eos_token = model.config.eos_token_id
device = "cuda:0"
max_length = 1000

In [21]:
#from Salminen et al.; a seed value was set to allow the generation to be re-produced and continued over multiple sessions
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

The generating loop

In [22]:
def generate(prompts):
  outputs = []
  for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    random_seed(seed_value=random_state, use_cuda=True)
    o = model.generate(**inputs, do_sample=do_sample, top_k=top_k, top_p=top_p, temperature=temperature, eos_token_id=eos_token, max_length=max_length)
    outputs.append(tokenizer.decode(o[0], skip_special_tokens=True))
  return outputs

In [23]:
aca_outputs = generate(aca_prompts)

In [24]:
fic_outputs = generate(fic_prompts)

In [25]:
news_outputs = generate(news_prompts)

In [26]:
aca_results = pd.DataFrame(aca_outputs, columns=['output'])
aca_results['prompt_category'] = 'ACA'
aca_results['prompt'] = aca_prompts

fic_results = pd.DataFrame(fic_outputs, columns=['output'])
fic_results['prompt_category'] = 'FIC'
fic_results['prompt'] = fic_prompts

news_results = pd.DataFrame(news_outputs, columns=['output'])
news_results['prompt_category'] = 'NEWS'
news_results['prompt'] = news_prompts

result_frames = [aca_results, fic_results, news_results]

result_df = pd.concat(result_frames, ignore_index=True)
result_df

,output,prompt_category,prompt
0,"In general, the file designer will choose a bu...",ACA,"In general, the file designer will choose a bu..."
1,The age distribution of a particular workforce...,ACA,The age distribution of a particular workforce...
2,Deixis has a powerful pragmatic base as most d...,ACA,Deixis has a powerful pragmatic base as most d...
3,9.8 Transport methods — vapour pressure osmome...,ACA,9.8 Transport methods — vapour pressure osmome...
4,We have termed this system the ‘Leicester scor...,ACA,We have termed this system the ‘Leicester score’.
5,"When this was applied to tissue sections, howe...",ACA,"When this was applied to tissue sections, howe..."
6,Inter-agency planning and collaboration The Un...,ACA,Inter-agency planning and collaboration The Un...
7,Our aim is first to determine the variation of...,ACA,Our aim is first to determine the variation of...
8,"In fact, it turns out that this solution can b...",ACA,"In fact, it turns out that this solution can b..."
9,A less elevated but similar claim was made by ...,ACA,A less elevated but similar claim was made by ...


In [27]:
result_df.to_csv('/content/drive/MyDrive/Linguistic Data Science/data/2epoch-academic-output.csv', index=False)